#### 在数据分析与挖掘中，通常会遇到众多的变量，这些变量之间往往具有一定的相关性。如果众多指标之间具有较强的相关性，不仅会增加计算复杂度，也会影响到模型的分析结果。一种思路就是把众多的变量转换为少数几个互不相关的综合变量，同时又不影响原来变量所反映的信息，即主成分分析

*  PCA的思想是将n维特征映射到k维上（k<n），这k维是全新的正交特征。这k维特征称为主成分，是重新构造出来的k维特征，而不是简单地从n维特征中去除其余n-k维特征

####  sklearn.decomposition 模块中可以导入 PCA主成分分析模块进行分析,具体的步骤

* 对原始数据进行标准化处理
* 计算样本相关系数矩阵
* 求相关系数矩阵的特征值和相应的特征向量
* 选择重要的主成分，并写出主成分表达式
* 计算主成分得分
* 依据主成分得分的数据，进一步从事统计分析

### 综合练习
* 利用主成分分析模型进行上市公司的综合评价，具体数据见表格。
* 总体规模指标：营业收入、营业利润、利润总额、净利润、资产总计、定资产净额 等
* 投资效率指标：净资产收益率、每股净资产、每股资本公积、每股收益  等

In [1]:
import pandas as pd
import tushare as ts
from sklearn.decomposition import PCA  # PCA主成分分析

In [2]:
data = pd.read_excel('./data/财务指标数据.xlsx')

In [4]:
data.columns 

Index(['股票代码', '股票简称', '现价(元)', '涨跌幅(%)', '营业收入(元)2019.12.31',
       '营业利润(元)2019.12.31', '利润总额(元)2019.12.31', '净利润(元)2019.12.31',
       '资产总计(元)2019.12.31', '固定资产净额(元)2019.12.31', '加权净资产收益率(%)2019.12.31',
       '每股净资产(元)2019.12.31', '每股资本公积(元)2019.12.31', '基本每股收益(元)2019.12.31',
       '营业总收入(元)2019.12.31', '预测营业利润 (元)2022.12.31', '预测营业利润 (元)2021.12.31',
       '预测营业利润 (元)2020.12.31', '净利润同比增长率(%)2019.12.31', '净资产(元)2019.12.31',
       '所属同花顺行业', '预测净资产收益率 (%)2022.12.31', '预测净资产收益率 (%)2021.12.31',
       '预测净资产收益率 (%)2020.12.31', '每股未分配利润(元)2019.12.31',
       '每股经营现金流(元)2019.12.31', '每股盈余公积(元)2019.12.31', '未分配利润(元)2019.12.31',
       'a股流通市值 (元)2021.01.29', '动态市盈率2021.01.29'],
      dtype='object')

In [5]:
new_columns = [i for i in data.columns  if '2019.12.31'in i] # 目标字段

In [6]:
new_columns = new_columns + ['股票代码','股票简称']  

In [7]:
data1 = data[new_columns]

In [10]:
data1 = data1.set_index('股票简称')

In [12]:
import numpy  as np
data1 = data1.apply(lambda x: x.replace('--',np.nan))

In [13]:
data2=data1.dropna() # 删除指标数据为空的股票

In [14]:
data2.shape

(3776, 18)

In [15]:
data2.dtypes

营业收入(元)2019.12.31        float64
营业利润(元)2019.12.31        float64
利润总额(元)2019.12.31        float64
净利润(元)2019.12.31         float64
资产总计(元)2019.12.31        float64
固定资产净额(元)2019.12.31      float64
加权净资产收益率(%)2019.12.31    float64
每股净资产(元)2019.12.31       float64
每股资本公积(元)2019.12.31      float64
基本每股收益(元)2019.12.31      float64
营业总收入(元)2019.12.31       float64
净利润同比增长率(%)2019.12.31    float64
净资产(元)2019.12.31         float64
每股未分配利润(元)2019.12.31     float64
每股经营现金流(元)2019.12.31     float64
每股盈余公积(元)2019.12.31      float64
未分配利润(元)2019.12.31       float64
股票代码                      object
dtype: object

In [19]:
X = data2.iloc[:,:-1] # 提取目标数据做为上市公司评价指标PCA建模数据

In [20]:
X.shape

(3776, 17)

In [21]:
normal_X = X.apply(lambda x : (x-x.mean())/x.std())

In [22]:
pca = PCA(n_components=0.95) # 主成分方差百分比累计0.95
Y = pca.fit_transform(normal_X) # 主成分的数据

In [23]:
Y.shape

(3776, 8)

In [24]:
weights = pca.explained_variance_ratio_  # 不同主成分数据的权重
weights

array([0.42625225, 0.20872896, 0.09007326, 0.07316032, 0.05821886,
       0.04595949, 0.03907259, 0.02300231])

In [25]:
score = np.dot(Y,weights)  # 根据权重和数据矩阵相乘得到对应的得分

In [27]:
ScoreD = pd.DataFrame(Y,index=normal_X.index ,columns=list(range(1,9)))
ScoreD['score'] = score

In [29]:
ScoreD.sort_values('score',ascending=False).iloc[0:10]

,1,2,3,4,5,6,7,8,score
股票简称,,,,,,,,,
工商银行,77.048037,-17.593450,-21.182284,0.535168,-1.246472,1.040758,-3.293499,-0.558262,27.134570
贵州茅台,22.573732,56.101600,-1.144686,3.459792,22.704456,-0.804473,5.085420,-1.845226,22.923257
建设银行,64.140908,-13.750816,-18.632835,0.551805,-0.505011,0.827779,-2.639602,-0.261064,22.731565
中国石油,45.439369,-9.605664,36.173017,-0.897560,-3.767023,-0.293567,1.141948,-0.803573,20.349540
中国石化,40.118864,-7.579303,42.639828,-1.026437,-3.376295,-0.825032,2.752075,1.411579,19.189868
农业银行,52.617223,-12.851541,-15.264195,0.200990,-1.062838,0.705700,-2.240951,-0.348664,18.260507
中国银行,50.809355,-12.803155,-12.763981,0.141511,-1.527225,0.345052,-0.746622,-1.996195,17.697728
中国平安,37.750944,5.998436,0.945824,1.444490,-0.255437,1.106885,-4.482547,8.690334,17.595099
招商银行,22.516320,2.157771,-5.491049,1.164474,1.493587,-0.704069,2.825790,-0.817979,9.784810
